In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# charts
import seaborn as sns 
import matplotlib.pyplot as plt
import squarify #TreeMap

# import graph objects as "go"
import plotly.graph_objs as go

# For exercise 4 and 5
import plotly.express as px

#for offline plotting using plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#for in-notebooks plot using plotly
init_notebook_mode(connected=True)

%matplotlib inline
from IPython.display import display

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#For Fuzzy Merge
from fuzzywuzzy import process

In [27]:
#Load CSV Files as Dataframes
a = pd.read_csv("Data/Transfermarkt_Cleaned.csv") 
b = pd.read_csv("Data/Stadium_Coordinates.csv")

In [16]:
transfermarkt_cleaned_df = a
stadium_coordinates = b

In [17]:
transfermarkt_cleaned_df[['Club Left','Club Joined']]

clubs_left_names = transfermarkt_cleaned_df['Club Left'].unique()
clubs_joined_names = transfermarkt_cleaned_df['Club Joined'].unique()

clubs_left_names = clubs_left_names.tolist()
clubs_joined_names = clubs_joined_names.tolist()

tm_club_names = np.unique(clubs_left_names + clubs_joined_names)
tm_club_names = tm_club_names.tolist()

df_club_names = pd.Series(tm_club_names)
df_club_names = df_club_names.to_frame()
df_club_names = df_club_names.rename(columns={0:'Club Name'})

df_club_names

,Club Name
0,1. FC Köln
1,1. FC Nürnberg
2,1. FC Union Berlin
3,1. FSV Mainz 05
4,1.FC Heidenheim
...,...
1033,Çaykur Rizespor
1034,Örebro SK
1035,Östersunds FK
1036,Újpest FC


In [18]:
stadium_coordinates['coordinates'] = stadium_coordinates['coordinates'].str.replace("Point\(","") 
stadium_coordinates['coordinates'] = stadium_coordinates['coordinates'].str.replace("\)","") 

temp = stadium_coordinates['coordinates'].str.split(expand = True)

temp = temp.rename(columns={0:'Longitude',1:'Latitude'})

a = temp[['Latitude','Longitude']].astype(float)

stadium_coordinates = stadium_coordinates.join(a)
stadium_coordinates_cleaned = stadium_coordinates.drop_duplicates(subset=['venueLabel'], keep="first", inplace=False)

stadium_coordinates_cleaned = stadium_coordinates_cleaned[['clubLabel','venueLabel','Latitude','Longitude']]

stadium_coordinates_cleaned

,clubLabel,venueLabel,Latitude,Longitude
0,S.S.C. Napoli,Stadio San Paolo,40.827967,14.193008
2,Manchester United F.C.,Old Trafford,53.463056,-2.291389
3,SD Compostela,Estadio Multiusos de San Lázaro,42.882767,-8.516014
6,Sacramento Knights,Sleep Train Arena,38.649167,-121.518056
7,Novara Calcio,Stadio Silvio Piola (Novara),45.435833,8.595833
...,...,...,...,...
9035,FK Smiltene/BJSS,Teperis stadium,57.430749,25.920292
9036,Pyrgos Limassol Women FC,Pyrgos Community Stadium,34.746644,33.188163
9037,Juventude Sport Clube,Sanches de Miranda Stadium,38.561944,-7.913889
9038,Louisville City FC,Lynn Family Stadium,38.259590,-85.733264


In [19]:
rows = []
for club in tm_club_names:
    highest = process.extractOne(club,stadium_coordinates_cleaned.clubLabel)
    print(highest)
    highest = list(highest)
    highest = highest + [club]
    rows.append(highest)

('1. FC Köln', 100, 81)
('1. FC Nürnberg', 100, 83)
('1. FC Union Berlin', 100, 1284)
('1. FSV Mainz 05', 100, 1394)
('1. FC Heidenheim', 97, 446)
('Cusco FC', 86, 52)
('A.A. Argentinos Juniors', 100, 1874)
('A.J. Auxerre', 100, 1585)
('A.C. Perugia Calcio', 86, 12)
('A.S. Saint-Étienne', 100, 784)
('A.C. Ajaccio', 91, 2802)
('AC Arles', 100, 3437)
('AC Bellinzona', 100, 873)
('AC Horsens', 100, 925)
('Le Havre AC', 95, 624)
('ACF Fiorentina', 100, 688)
('A.C.R. Messina', 88, 70)
('FC Forex Brașov', 64, 6834)
('ADO Den Haag', 100, 903)
('A.E.K. Athens F.C.', 86, 311)
('AEL Limassol FC', 95, 3411)
('AFC Ajax', 100, 353)
('AFC Atletico Vaslui', 86, 447)
('AFC Bournemouth', 100, 151)
('AFC Bournemouth', 100, 151)
('Nîmes Olympique', 54, 1664)
('AIK Fotboll', 100, 2193)
('Orfeas Xanthi F.C.', 86, 7221)
('Olympiakos Nicosia FC', 86, 4933)
('AS Monaco FC', 100, 1175)
('AS Monaco FC', 86, 1175)
('ASA 2013 Târgu Mureș', 76, 6326)
('AZ Alkmaar', 100, 1141)
('AZ Alkmaar', 87, 1141)
('Aalesunds F

In [24]:
df = pd.DataFrame(rows, columns=["Club Name Wikidata","random1","random2","Club Name TM"])

merged_df = pd.merge(df,stadium_coordinates_cleaned,
                    how='inner',
                    right_on = 'clubLabel',
                    left_on='Club Name Wikidata')

merged_df = merged_df[['Club Name TM','Club Name Wikidata','venueLabel','Latitude','Longitude']]

merged_df.to_csv("club_stadium_coordinates_Final.csv",index=False)

,Unnamed: 0,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,Market Valuation,Transfer,Surplus/Deficit,Year
0,0,0,1/1/2010,Douglas Costa,19,Right Winger,Brazil,Grêmio Esportivo Brasiliense,Série A,FC Shakhtar Donetsk,NaN,4000000,8000000.0,4000000.0,2010
1,1,1,1/1/2010,Florent Sinama-Pongolle,25,Second Striker,France,Atlético Madrid,LaLiga,Sporting CP,NaN,7000000,6500000.0,-500000.0,2010
2,2,2,1/1/2010,Alex Teixeira,19,Left Winger,Brazil,CR Vasco da Gama,Série A,FC Shakhtar Donetsk,NaN,3500000,6000000.0,2500000.0,2010
3,3,3,1/1/2010,Keisuke Honda,23,Attacking Midfield,Japan,VVV-Venlo,Eredivisie,PFC CSKA Moscow,NaN,3000000,6000000.0,3000000.0,2010
4,4,4,1/1/2010,Younès Kaboul,23,Centre-Back,France,Portsmouth F.C.,Premier League,Tottenham Hotspur F.C.,NaN,5700000,5900000.0,200000.0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7501,61758,63178,31/8/2007,Esben Hansen,26,Defensive Midfield,Denmark,Odense Boldklub,Superligaen,1.FC K'lautern,NaN,500000,200000.0,-300000.0,2007
7502,61767,63187,1/9/2007,Bonaventure Kalou,29,Right Winger,Cote d'Ivoire,Lens,Ligue 1,Al Jazira Club,NaN,2000000,1500000.0,-500000.0,2007
7503,61770,63190,1/9/2007,Gabriel Bostina,30,Left Midfield,Romania,FC Steaua Bucuresti,Liga 1,Dinamo Bukarest,NaN,250000,100000.0,-150000.0,2007
7504,61792,63212,3/9/2007,Hakan Balta,24,Left-Back,Turkey,Manisaspor,Süper Lig,Galatasaray S.K.,NaN,2000000,1000000.0,-1000000.0,2007
